In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
import uuid
import time

from joblib import dump, load

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import balanced_accuracy_score

from collections import defaultdict

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from unidecode import unidecode

nltk.download('punkt') ## algorithm for tokenization
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/david/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
train = pd.read_csv('data/train.csv')
unreliable_weight = 0.4
train['weight'] = train['label_quality'].apply(lambda quality: 1. if quality == 'reliable' else unreliable_weight)

In [3]:
train_spanish = train[train.language == 'spanish']
train_portuguese = train[train.language == 'portuguese']

In [4]:
train_spanish_reliable = train_spanish[train_spanish.label_quality == 'reliable']
train_portuguese_reliable = train_portuguese[train_portuguese.label_quality == 'reliable']

In [5]:
def filter_small_categories(df):
    grouped = df.groupby(['category']).count()
    available_categories = list(grouped[grouped['title'] > 1].index.to_numpy())
    filtered = df[df.category.isin(available_categories)]
    return filtered

In [6]:
def my_preprocessor(doc):
    return re.sub('[0-9¡¨ª®°´·º»½¿' + string.punctuation + ']', '', unidecode(doc)).lower()

spanish_stemmer = SnowballStemmer('spanish')
portuguese_stemmer = SnowballStemmer('portuguese')

def spanish_stemmer_tokenizer(doc):
    tokens = word_tokenize(doc)
    return [spanish_stemmer.stem(token) for token in tokens]

def portuguese_stemmer_tokenizer(doc):
    tokens = word_tokenize(doc)
    return [portuguese_stemmer.stem(token) for token in tokens]

In [7]:
def elapsed_time(start, message):
    end = time.time()
    print( message + ': ', end-start )

In [8]:
def train(titles, labels, weights, language='spanish', batches=200, epochs=400, max_features=8000, 
          min_df=3, max_df=0.7, test_size=0.1):
    titles_train, titles_test, y_train, y_test, w_train, w_test = train_test_split(titles, labels, weights, test_size=test_size, random_state=42, stratify=labels)
    print(titles_train.shape, titles_test.shape, y_train.shape, y_test.shape)
    
    run_id = uuid.uuid4().hex
    print('Run id: ', run_id)
    
    if language == 'spanish':
        stop_words = list(map(lambda word: spanish_stemmer.stem(unidecode(word)), stopwords.words(language)))
        vectorizer = CountVectorizer(max_features=max_features, max_df=max_df, min_df=min_df, strip_accents='unicode', 
                                        stop_words=stop_words, tokenizer=spanish_stemmer_tokenizer, 
                                        preprocessor=my_preprocessor)
    else:
        stop_words = list(map(lambda word: portuguese_stemmer.stem(unidecode(word)), stopwords.words(language)))
        vectorizer = CountVectorizer(max_features=max_features, max_df=max_df, min_df=min_df, strip_accents='unicode', 
                                        stop_words=stop_words, tokenizer=portuguese_stemmer_tokenizer, 
                                        preprocessor=my_preprocessor)
    
    start_time = time.time()
    vectorizer.fit(titles_train)
    elapsed_time(start_time, 'Fit vectorizer')
    
    dump(vectorizer, 'models/' + language + '_vectorizer_' + run_id + '.joblib')
    
    tokens = vectorizer.get_feature_names()

    start_time = time.time()
    X_train = vectorizer.transform(titles_train)
    elapsed_time(start_time, 'Word2Vec X_train')
    
    start_time = time.time()
    X_test = vectorizer.transform(titles_test)
    elapsed_time(start_time, 'Word2Vec X_test')
    
    samples = X_train.shape[0]
    print('samples per batch: ', samples//batches)

    classifier = MultinomialNB()
    
    categories = y_train.unique()

    for epoch in range(epochs):
        
        for batch in range(batches):
            start_time = time.time()
            start = (samples * batch)//batches
            end = (samples * (batch + 1))//batches
            classifier.partial_fit(X_train[ start:end ], y_train[ start:end ], 
                                   sample_weight=w_train[ start:end ], classes=categories)
            if(batch==0):
                elapsed_time(start_time, 'Batch')
        if epoch % 5 == 0:
            y_predicted = classifier.predict(X_test)
            score = balanced_accuracy_score(y_true=y_test, y_pred=y_predicted, sample_weight=w_test)
            print('finished epoch: ', epoch, 'with score:', score)
            dump(classifier, 'models/checkpoint/' + language + '_classifier_' + run_id + '_' + str(epoch) + '.joblib') 

    y_predicted = classifier.predict(X_test)
    score = balanced_accuracy_score(y_true=y_test, y_pred=y_predicted, sample_weight=w_test)
    print('Final score: ', score)

    dump(classifier, 'models/' + language + '_classifier_' + run_id + '.joblib') 

# Portuguese

In [9]:
filtered = filter_small_categories(train_portuguese)

train(filtered['title'], filtered['category'], filtered['weight'], language='portuguese', max_features = 20000,
     batches=800, epochs=60, test_size=0.002)

(9980000,) (20000,) (9980000,) (20000,)
Run id:  cb7e1220a901413aa002e26667cf29e6


/home/david/miniconda3/envs/ml-challenge/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['dep'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Fit vectorizer:  2419.16743850708
Word2Vec X_train:  2273.612967967987
Word2Vec X_test:  4.434809923171997
samples per batch:  12475
Batch:  1.2439532279968262
finished epoch:  0 with score: 0.7169444564409991
Batch:  1.057809591293335
Batch:  1.0532116889953613
Batch:  1.0431883335113525
Batch:  1.0373973846435547
Batch:  1.056246042251587


/home/david/miniconda3/envs/ml-challenge/lib/python3.5/site-packages/sklearn/metrics/classification.py:1745: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


finished epoch:  5 with score: 0.7905628376022475
Batch:  1.2663767337799072
Batch:  1.0384061336517334
Batch:  1.0395982265472412
Batch:  1.0424039363861084
Batch:  1.0501832962036133


...

finished epoch:  50 with score: 0.8169656068289127
Batch:  1.697082757949829
Batch:  1.0552968978881836
Batch:  1.0593597888946533
Batch:  1.0440232753753662
Batch:  1.061199426651001


/home/david/miniconda3/envs/ml-challenge/lib/python3.5/site-packages/sklearn/metrics/classification.py:1745: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


finished epoch:  55 with score: 0.8171818491741308
Batch:  1.620112657546997
Batch:  1.0542919635772705
Batch:  1.0568559169769287
Batch:  1.0579240322113037


/home/david/miniconda3/envs/ml-challenge/lib/python3.5/site-packages/sklearn/metrics/classification.py:1745: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


Final score:  0.8171494065095651


# Spanish

In [10]:
filtered = filter_small_categories(train_spanish)

train(filtered['title'], filtered['category'], filtered['weight'], language='spanish', max_features = 20000,
     batches=800, epochs=60, test_size=0.002)

(9980000,) (20000,) (9980000,) (20000,)
Run id:  0495c1663b1442518c1282fa490a8be2


/home/david/miniconda3/envs/ml-challenge/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['estuvi', 'hubi', 'tuvi'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Fit vectorizer:  2160.6638283729553
Word2Vec X_train:  2164.277093410492
Word2Vec X_test:  4.3777549266815186
samples per batch:  12475
Batch:  1.604860782623291
finished epoch:  0 with score: 0.6894358133677794
Batch:  1.6023211479187012
Batch:  1.0592050552368164
Batch:  1.0514397621154785
Batch:  1.0554003715515137
Batch:  1.0685057640075684
finished epoch:  5 with score: 0.7703840962260698
Batch:  1.3756449222564697
Batch:  1.0657825469970703
Batch:  1.0580158233642578
Batch:  1.0552031993865967
Batch:  1.0449573993682861
finished epoch:  10 with score: 0.7873343422612727
Batch:  1.2690224647521973
Batch:  1.0672268867492676
Batch:  1.0845885276794434
Batch:  1.0503854751586914
Batch:  1.0615041255950928


/home/david/miniconda3/envs/ml-challenge/lib/python3.5/site-packages/sklearn/metrics/classification.py:1745: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


finished epoch:  15 with score: 0.7934842372674912
Batch:  1.2656989097595215
Batch:  1.0597023963928223
Batch:  1.0645697116851807
Batch:  1.0622332096099854
Batch:  1.0587666034698486


...

finished epoch:  50 with score: 0.8002197057603094
Batch:  1.380624771118164
Batch:  1.0443518161773682
Batch:  1.082435131072998
Batch:  1.053795337677002
Batch:  1.0418567657470703


/home/david/miniconda3/envs/ml-challenge/lib/python3.5/site-packages/sklearn/metrics/classification.py:1745: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


finished epoch:  55 with score: 0.8003359171533589
Batch:  1.2525625228881836
Batch:  1.0906317234039307
Batch:  1.0672686100006104
Batch:  1.0749897956848145


/home/david/miniconda3/envs/ml-challenge/lib/python3.5/site-packages/sklearn/metrics/classification.py:1745: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


Final score:  0.8003615999313354
